In [1]:
import pandas as pd
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

    



In [2]:
# Load the validation dataset
from utilities.misc import load_bayes_risk_dataframe, map_to_utility
n_hypothesis = 100
n_references = 1000
utility = "comet"

df = load_bayes_risk_dataframe('ancestral', 100, 1000, 'test', utility, 0, prepend='..' )
df["utility"] = df[["utilities", 'references_count']].apply(map_to_utility, axis=1)

#df = df.explode(['hypotheses', 'utilities', 'utility'])

C:\Users\gerso\anaconda3\envs\thesis\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
df

,source,target,hypotheses,references_count,utilities,utility
0,Ich bin nicht krank.,I'm not sick.,"[I'm not ill., I'm not sick., I am not sick., ...","[80, 799, 99, 14, 1, 1, 1, 2, 1, 1, 1]","[[0.7963573, 0.9171567, 1.23882, 1.0933249, 0....","[0.9397974753528834, 1.2060345169901847, 1.121..."
1,Tom wollte Schriftsteller werden.,Tom wanted to be a writer.,"[Tom wanted to be a writer., Tom wanted to be ...","[605, 272, 45, 1, 29, 7, 5, 2, 1, 2, 4, 2, 3, ...","[[1.2125007, 1.0591949, 0.97408587, 0.48393297...","[1.1293036549687385, 0.9209178121536971, 1.112..."
2,Welches Hauptfach gedenkst du zu studieren?,What do you plan to major in in college?,"[What do you usually plan to major in?, What i...","[1, 4, 1, 3, 1, 7, 6, 1, 1, 4, 39, 3, 219, 2, ...","[[-0.018019106, 0.28967944, -0.92702985, 0.357...","[0.24660021169669927, -0.5506506634764373, -0...."
3,"Es gibt nur ein Tempo, und das ist das richtige.","There's only one speed, and that's the right one.","[There's only one speed, and that's the right....","[1, 1, 3, 28, 1, 1, 1, 1, 1, 6, 5, 1, 1, 1, 1,...","[[0.46352598, -0.18130341, 0.2582156, 0.850075...","[0.13661964124068618, -0.18260218991246074, -0..."
4,"Ich werde entscheiden, ob es besser ist oder n...",I will judge whether it is superior or not.,"[I'll decide whether it's better or not., I'll...","[1, 278, 332, 7, 3, 118, 1, 8, 1, 1, 1, 1, 1, ...","[[0.42733178, 1.0905064, 0.9884072, 0.7873084,...","[0.9108508883211762, 0.9486991599481552, 0.721..."
...,...,...,...,...,...,...
2495,Ich empfinde keine persönliche Abneigung gegen...,I have no personal hostility to the system.,[I don't feel any personal entrance to the sys...,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 8, 1, ...","[[0.2029506, 0.10191179, -0.3412029, -0.186559...","[-0.0891167614441365, -0.28414561685733497, -1..."
2496,"Es sieht so aus, als ob es bald aufklaren wird.",It looks like it's going to clear up soon.,"[It looks like it will clear up soon., It look...","[567, 96, 32, 1, 1, 9, 59, 6, 1, 40, 2, 1, 7, ...","[[1.0305676, 1.1278272, 0.90105724, 1.019571, ...","[0.9587589915841818, 0.8566607671845704, 1.005..."
2497,Ich muss gleich das Badezimmer putzen.,I must clean the bathroom right away.,"[I must clean the bathroom right away., I have...","[75, 274, 9, 1, 24, 246, 8, 113, 8, 1, 1, 14, ...","[[0.91010875, 1.1603966, 1.0014539, 0.32372063...","[0.968125136949122, 0.9725559178963303, 0.9595..."
2498,"Tom sagte Maria, dass er Elke niedlich finde.",Tom told Mary that he thought Alice was cute.,[Tom told Mary that he thought Alice was cute.],"[995, 4, 1]","[[1.1222363, 1.0428538, 0.99162143]]",[1.1217881073355676]


In [4]:
import numpy as np
def get_m_mc_estimate(x, m=1):
    utilities_count = x["references_count"]
    p = utilities_count / np.sum(utilities_count)
    indices = np.random.choice(len(utilities_count), size=m, p=p)
    weights = utilities_count[indices]
    avg_utils = []
    for util in x["utilities"]:
        avg_utils.append(np.sum(weights * util[indices]) / np.sum(weights))
    return avg_utils
        

In [6]:
ms = [
    1,2,3,4,5,10, 25, 50, 100
]
for m in ms:
    df["{}_mc_estimate".format(m)] = df.apply(lambda x: get_m_mc_estimate(x, m), axis=1)

In [7]:
df

,source,target,hypotheses,references_count,utilities,utility,1_mc_estimate,2_mc_estimate,3_mc_estimate,4_mc_estimate,5_mc_estimate,10_mc_estimate,25_mc_estimate,50_mc_estimate,100_mc_estimate
0,Ich bin nicht krank.,I'm not sick.,"[I'm not ill., I'm not sick., I am not sick., ...","[80, 799, 99, 14, 1, 1, 1, 2, 1, 1, 1]","[[0.7963573, 0.9171567, 1.23882, 1.0933249, 0....","[0.9397974753528834, 1.2060345169901847, 1.121...","[0.9171566963195801, 1.250632643699646, 1.1349...","[0.9171566963195801, 1.250632643699646, 1.1349...","[0.9171566963195801, 1.250632643699646, 1.1349...","[0.9299149746791675, 1.2409738345214953, 1.127...","[0.9171566963195801, 1.250632643699646, 1.1349...","[0.9205322417471257, 1.245148746903094, 1.1335...","[0.92058546216459, 1.248036854982697, 1.132880...","[0.9227623268134245, 1.2449096721974213, 1.131...","[0.9205301448419368, 1.2463505363219327, 1.133..."
1,Tom wollte Schriftsteller werden.,Tom wanted to be a writer.,"[Tom wanted to be a writer., Tom wanted to be ...","[605, 272, 45, 1, 29, 7, 5, 2, 1, 2, 4, 2, 3, ...","[[1.2125007, 1.0591949, 0.97408587, 0.48393297...","[1.1293036549687385, 0.9209178121536971, 1.112...","[1.2125006914138794, 0.9418765902519226, 1.106...","[1.2125006914138794, 0.9418765902519226, 1.106...","[1.1399172812155582, 0.9024879236756665, 1.154...","[1.2125006914138794, 0.9418765902519226, 1.106...","[1.1771474322710769, 0.922691521198884, 1.1298...","[1.1572818847512014, 0.9119089256513062, 1.143...","[1.1754637384829498, 0.9255573859927718, 1.126...","[1.1793707223690526, 0.9268948853518225, 1.125...","[1.1803301577306204, 0.9257134126192713, 1.126..."
2,Welches Hauptfach gedenkst du zu studieren?,What do you plan to major in in college?,"[What do you usually plan to major in?, What i...","[1, 4, 1, 3, 1, 7, 6, 1, 1, 4, 39, 3, 219, 2, ...","[[-0.018019106, 0.28967944, -0.92702985, 0.357...","[0.24660021169669927, -0.5506506634764373, -0....","[0.09653334319591522, -0.5465942621231079, -0....","[0.5649292529383793, -0.3886766916588892, 0.03...","[0.5376828072907088, -0.34402750258321885, 0.0...","[0.5499683351398589, -0.3453270543265987, 0.02...","[0.5367479577128377, -0.3421365582450692, 0.01...","[0.5558076715046307, -0.3449970236280512, 0.03...","[0.5533552239522271, -0.3516147514747134, 0.03...","[0.5571418972208332, -0.37441020392072505, 0.0...","[0.5595157477022247, -0.41718955223822635, 0.0..."
3,"Es gibt nur ein Tempo, und das ist das richtige.","There's only one speed, and that's the right one.","[There's only one speed, and that's the right....","[1, 1, 3, 28, 1, 1, 1, 1, 1, 6, 5, 1, 1, 1, 1,...","[[0.46352598, -0.18130341, 0.2582156, 0.850075...","[0.13661964124068618, -0.18260218991246074, -0...","[0.6841135621070862, -0.1310815066099167, 0.13...","[0.10352236777544022, -0.1156981885433197, 0.1...","[-0.21211480100949606, -0.4784037371476491, -0...","[0.7613151528392778, 0.11566124969359601, 0.01...","[0.07053407430648803, -0.022805917263031005, 0...","[0.46654192900115793, -0.25982907016507606, -0...","[0.30487475519043367, -0.0650403809708518, -0....","[0.4171083295831875, -0.10431183574303073, -0....","[0.44456094269160973, -0.07023897662175575, -0..."
4,"Ich werde entscheiden, ob es besser ist oder n...",I will judge whether it is superior or not.,"[I'll decide whether it's better or not., I'll...","[1, 278, 332, 7, 3, 118, 1, 8, 1, 1, 1, 1, 1, ...","[[0.42733178, 1.0905064, 0.9884072, 0.7873084,...","[0.9108508883211762, 0.9486991599481552, 0.721...","[1.0905064344406128, 1.0002769231796265, 0.755...","[0.987694629856774, 1.0959358628805693, 0.7916...","[1.0826428338885308, 0.9938784354499408, 0.750...","[0.9869359912457051, 1.095368491412999, 0.7908...","[1.0334071442410868, 1.0311084064139866, 0.768...","[1.0305315120039276, 0.9974460448959828, 0.758...","[1.0369801921166097, 1.044853253562365, 0.7728...","[1.0231669691543845, 1.0377938751112281, 0.771...","[1.0227486544025357, 1.0492467216843941, 0.774..."
...,...,...,...,...,...,...,...,...,...,...,...,.

In [8]:
from utilities.misc import load_bayes_risk_dataframe, map_to_utility

df["utility"] = df[["utilities", 'references_count']].apply(map_to_utility, axis=1)


In [9]:
from comet import download_model, load_from_checkpoint
from utilities.wrappers.CometWrapper import CometWrapper

# Get the comet score model
model_path = download_model("wmt20-comet-da")
model = load_from_checkpoint(model_path)

model.to("cuda")
model.eval()
wrapped_model = CometWrapper(model)

wmt20-comet-da is already in cache.
Some weights of the model checkpoint at xlm-roberta-large were not used when initializing XLMRobertaModel: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Encoder model frozen.


In [ ]:
from utilities.evaluation import *
from pathlib import Path
import json
base_ref = '../results/'

for m in ms:
    summary = {
        
    }
    name = '{}_mc_estimate'.format(m)
    pretty_name = '{}-MC Estimate'.format(m)
    save_location = base_ref + name + '/'
    Path(save_location).mkdir(parents=True, exist_ok=True)
    df["predictions"] = df[name]
    
    plot_rank_vs_predicted_util(df,save_location)
    
    kendall_taus, lengths, one_hyp_count = plot_kendall_taus(df, save_location, pretty_name )
    
    summary["mean_kendall_taus"] = np.mean(kendall_taus)
    summary["median_kendall_taus"] = np.median(kendall_taus)
    summary["std_kendall_taus"] = np.std(kendall_taus)
    
    targets = df["target"].to_list()
    sources = df["source"].to_list()
    
    
    best_predictions = get_highest_scoring_predictions(df)
    top_10_percent_prediction = get_top_p_predictions(df, 0.1)
 
    
    top_10_comet_scores = wrapped_model.batch_predict(sources, top_10_percent_prediction, targets)
    
    
    #sns.histplot(top_10_comet_scores)
    
    summary["top_10_comet_mean"] = np.mean(top_10_comet_scores)
    summary["top_10_comet_median"] = np.median(top_10_comet_scores)
    summary["top_10_comet_std"] = np.std(top_10_comet_scores)
    
    best_predictions_comet_scores = wrapped_model.batch_predict(sources, best_predictions, targets)
    
    summary["best_comet_mean"] = np.mean(best_predictions_comet_scores)
    summary["best_comet_median"] = np.median(best_predictions_comet_scores)
    summary["best_comet_std"] = np.std(best_predictions_comet_scores)
    
    
    
    # Save the summary
    summary_ref = save_location + 'summary.json'

    with open(summary_ref, 'w') as fp:
        json.dump(summary, fp)
    

In [ ]:
def print_as_table(values, )
    result = ''

    for m in ms:
        name = '{}_mc_estimate'.format(m)
        pretty_name = str(m)

        result_ref = '../results/{}/'.format(name)
        summary_ref = result_ref + 'summary.json'
        with open(summary_ref, 'r') as f: 
            data = json.load(f)
            result += pretty_name
            for val in values:
                result += ' & ' + "{:1e}".format(data[val])
            result += ' \\\\ \n'

In [ ]:
values = [
#     'MSE',
    'best_comet_mean',
     'top_10_comet_mean',
#     'top_10_comet_median',
#     'top_10_comet_std',

#     'mean_kendall_taus',
#     'median_kendall_taus',
#     'std_kendall_taus',
]


        
print_as_table(["median_kendall_taus", "mean_kendall_taus"])

print_as_table(["MSE"])

print_as_table(["best_comet_mean", "top_10_comet_mean", ])